In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer
#from nltk.metrics import edit_distance
from nltk.stem.porter import *
stemmer = PorterStemmer()
#from nltk.stem.snowball import SnowballStemmer #0.003 improvement but takes twice as long as PorterStemmer
#stemmer = SnowballStemmer('english')
import re
#import enchant
import random
random.seed(2016)
from autocorrect import spell
import enchant



##### Step 1: Load the data

In [2]:
start_time = time.time()
df_train = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/test.csv', encoding="ISO-8859-1")[:1000] #update here
df_pro_desc = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/product_descriptions.csv')[:1000] #update here
df_attr = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/attributes.csv')


In [3]:
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
num_train = df_train.shape[0]
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')
print("--- Files Loaded: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- Files Loaded: 0.12 minutes ---


In [4]:
f = open('spelling.txt','r')
zspell = {}
for line in f:
    a, b = line.strip("\n").split("|")
    zspell[a]=b
f.close()
zspell

def str_stem(s): 
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = re.sub(r"([0-9])( *),( *)([0-9])", r"\1\4", s)
        s = s.replace(","," ")
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = s.replace("   "," ")
        s = s.replace("  "," ").strip(" ")
        s = re.sub(r"(.*)\.$", r"\1", s) #end period
        s = re.sub(r"(.*)\/$", r"\1", s) #end period
        s = re.sub(r"^\.(.*)", r"\1", s) #start period
        s = re.sub(r"^\/(.*)", r"\1", s) #start slash
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        #s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        s = s.lower()
        s = (" ").join([str(zspell[z]) if z in zspell else z for z in s.split(" ")])
        return s
    else:
        return "null"

In [18]:
def seg_words(str1, str2):
    str2 = str2.lower()
    str2 = re.sub("[^a-z0-9./]"," ", str2)
    str2 = [z for z in set(str2.split()) if len(z)>2]
    words = str1.lower().split(" ")
    s = []
    for word in words:
        if len(word)>3:
            s1 = []
            s1 += segmentit(word,str2,True)
            if len(s)>1:
                s += [z for z in s1 if z not in ['er','ing','s','less'] and len(z)>1]
            else:
                s.append(word)
        else:
            s.append(word)
    return (" ".join(s))

def segmentit(s, txt_arr, t):
    st = s
    r = []
    for j in range(len(s)):
        for word in txt_arr:
            if word == s[:-j]:
                r.append(s[:-j])
                #print(s[:-j],s[len(s)-j:])
                s=s[len(s)-j:]
                r += segmentit(s, txt_arr, False)
    if t:
        i = len(("").join(r))
        if not i==len(st):
            r.append(st[i:])
    return r

def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

###### Step 2 : Stemming

In [9]:
start_time = time.time()
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))
print("--- Stemming: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- Stemming: 2.15 minutes ---


In [13]:
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
print("--- Prod Info: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- Prod Info: 4.54 minutes ---


In [19]:
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)
df_all['search_term'] = df_all['product_info'].map(lambda x:seg_words(x.split('\t')[0],x.split('\t')[1]))

In [21]:
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))

In [23]:
df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))

In [24]:
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']
df_brand = pd.unique(df_all.brand.ravel())
d={}
i = 1000
for s in df_brand:
    d[s]=i
    i+=3
df_all['brand_feature'] = df_all['brand'].map(lambda x:d[x])
df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(x))
df_all.to_csv('df_all.csv')

In [6]:
df_all['product_description'][1]

'Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a "Z" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws'

In [10]:
df_all['product_description'][1]

'not onli do angl make joint stronger they also provid more consist straight corner simpson strong tie offer a wide varieti of angl in variou size and thick to handl light duti job or project where a structur connect is need some can be bent (skewed) to match the project for outdoor project or those where moistur is present use our zmax zinc coat connector which provid extra resist against corros (look for a "z" at the end of the model number) versatil connector for variou 90 connect and home repair project stronger than angl nail or screw fasten alon help ensur joint are consist straight and strong dimensions: 3in. xbi 3in. xbi 1 1/2in. made from 12 gaug steel galvan for extra corros resist instal with 10 d common nail or #9 xbi 1 1/2in. strong drive sd screw'

In [20]:
df_all['search_term'][1]

'l bracket'

In [12]:
xyz[1]

'l bracket\tsimpson strong tie 12 gaug angl\tnot onli do angl make joint stronger they also provid more consist straight corner simpson strong tie offer a wide varieti of angl in variou size and thick to handl light duti job or project where a structur connect is need some can be bent (skewed) to match the project for outdoor project or those where moistur is present use our zmax zinc coat connector which provid extra resist against corros (look for a "z" at the end of the model number) versatil connector for variou 90 connect and home repair project stronger than angl nail or screw fasten alon help ensur joint are consist straight and strong dimensions: 3in. xbi 3in. xbi 1 1/2in. made from 12 gaug steel galvan for extra corros resist instal with 10 d common nail or #9 xbi 1 1/2in. strong drive sd screw'

--- Stemming: 6.76 minutes ---
--- Prod Info: 6.76 minutes ---
--- Len of: 6.77 minutes ---
--- Query In: 6.86 minutes ---
--- Query Last Word In: 6.88 minutes ---
--- Features Set: 6.98 minutes ---


In [4]:
df_pro_desc['product_description'][9]

'Valley View Industries Metal Stakes (4-Pack) are 9 in. galvanized steel stakes for use with all Valley View lawn edgings and brick and paver edgings. These utility stakes can also be used for many other purposes. It is recommended that anchor stakes are used every five feet on designs that have the edging in straight lengths. Where there are curved designs for edgings, additional anchor stakes are recommended at the curve points. Anchor stakes should be staked in at a 45 degree angle. Gloves and eye protection are recommended.Can be used with all valley View lawn edgings and brick/ paver edgingsUtility stakes can be used for many purposesGalvanized steel for strength9 in. lengthPriced competitively yet provides much more value in product'

In [6]:
df_pro_desc['product_description'][9]

'valley view industries metal stakes (4 pack) are 9 in galvanized steel stakes for use with all valley view lawn edgings and brick and paver edgings these utility stakes can also be used for many other purposes it is recommended that anchor stakes are used every five feet on designs that have the edging in straight lengths where there are curved designs for edgings additional anchor stakes are recommended at the curve points anchor stakes should be staked in at a 45 degree angle gloves and eye protection are recommended can be used with all valley view lawn edgings and brick paver edgings utility stakes can be used for many purposes galvanized steel for strength 9 in length priced competitively yet provides much more value in product'

#### PART 2: Feature Creation

In [34]:
import time
start_time = time.time()

import numpy as np
import pandas as pd
from nltk.metrics import edit_distance
import re

def str_common_word(str1, str2):
    str2 = str2.lower().split(" ")
    if str1 in str2:
        cnt=1
    else:
        cnt=0
    return cnt

def str_common_word2(str1, str2):
    str2 = str(str2).lower()
    if str2.find(str1)>=0:
        cnt=1
    else:
        cnt=0
    return cnt

In [35]:
start_time = time.time()
df_train = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/test.csv', encoding="ISO-8859-1")#update here
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = df_all[['product_uid','search_term','product_title']]
df_all.reset_index(inplace=True)

In [36]:
df_all.head()

,index,product_uid,search_term,product_title
0,0,100001,angle bracket,Simpson Strong-Tie 12-Gauge Angle
1,1,100001,l bracket,Simpson Strong-Tie 12-Gauge Angle
2,2,100002,deck over,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...
3,3,100005,rain shower head,Delta Vero 1-Handle Shower Only Faucet Trim Ki...
4,4,100005,shower only faucet,Delta Vero 1-Handle Shower Only Faucet Trim Ki...


In [37]:
df_prod = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/product_descriptions.csv').fillna(" ") #update here
df_attr = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/attributes.csv').fillna("")


In [38]:
d_prod_query = {}
for i in range(len(df_all)):
    b_ = str(df_all['product_uid'][i])
    if b_ not in d_prod_query:
        d_prod_query[b_] = [list(set(str(df_all['search_term'][i]).lower().split(" "))), 
                            str(df_all['product_title'][i]).lower(),
                            str(df_prod.loc[df_prod['product_uid'] == df_all['product_uid'][i]]['product_description'].iloc[0]).lower()]
    else:
        d_prod_query[b_][0] = list(set(d_prod_query[b_][0] + list(set(str(df_all['search_term'][i]).lower().split(" ")))))

f = open("dict.txt", "w")
f.write(str(d_prod_query))
f.close()